# Web Scraping LinkedIn Job Postings (Authenticated LinkedIn Account)

## File Imports and Set-Up

In [1]:
pip install install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
import time
import pandas as pd
import os

#using regular expressions to find the first number
import re

from dotenv import load_dotenv

#loading env variables
load_dotenv()

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

url1 = 'https://www.linkedin.com/jobs/search/?currentJobId=3705320512&f_E=2&f_WT=2&keywords=flutter&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true'

driver = webdriver.Chrome()
driver.get(url1)
driver.implicitly_wait(10)

## Logging in to LinkedIn Using a Dummy Account

In [3]:
signin_button = driver.find_element(By.CLASS_NAME, 'nav__button-secondary')
signin_button.click()

username = driver.find_element(By.ID, 'username')
username.send_keys(os.environ.get('EMAIL'))

password = driver.find_element(By.ID, 'password')
password.send_keys(os.environ.get('PASSWORD'))

login_button = driver.find_element(By.CLASS_NAME, 'btn__primary--large')
login_button.click()

## Counting the Number of Jobs

In [4]:
y = driver.find_elements(By.CLASS_NAME, 'jobs-search-results-list__subtitle')[0].text

try:
    if y.endswith('+') and "," in y:
        cleaned_y = y[:-1].replace(",", "")

    else:
        cleaned_y = y
        
    n = pd.to_numeric(cleaned_y)
    
except:
    parts = y.split()

    n = int(parts[0])

n

34

## Collecting the Links for Each Job Posting

In [5]:
# Get all links for these offers
links = []

# Navigate 13 pages
print('Links are being collected now.')
try:
    for page in range(2, 16):
        time.sleep(2)
        jobs_block = driver.find_element(By.CLASS_NAME, 'jobs-search-results-list')
        jobs_list = jobs_block.find_elements(By.CSS_SELECTOR, '.jobs-search-results__list-item')

        for job in jobs_list:
            all_links = job.find_elements(By.TAG_NAME, 'a')
            for a in all_links:
                href = a.get_attribute('href')
                if href and href.startswith("https://www.linkedin.com/jobs/view") and href not in links:
                    links.append(href)

            # scroll down for each job element
            driver.execute_script("arguments[0].scrollIntoView();", job)

        print(f'Collecting the links on page: {page - 1}')
        # go to next page:
        next_page_button = driver.find_element(By.XPATH, f"//button[@aria-label='Page {page}']")
        driver.execute_script("arguments[0].click();", next_page_button)
        time.sleep(3)
except Exception as e:
    print(e)

print(f'Found {len(links)} links for job offers')

Links are being collected now.
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@aria-label='Page 3']"}
  (Session info: chrome=118.0.5993.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104301e08 chromedriver + 5025288
1   chromedriver                        0x00000001042f8c23 chromedriver + 4987939
2   chromedriver                        0x0000000103e9ae67 chromedriver + 409191
3   chromedriver                        0x0000000103eea1b9 chromedriver + 733625
4   chromedriver                        0x0000000103eea371 chromedriver + 734065
5   chromedriver                        0x0000000103f30194 chromedriver + 1020308
6   chromedriver                        0x0000000103f1150d chromedriver + 894221
7   chromedriver                        0x0000000103f2d571 chromedriver + 1009009
8   c

## Scraping Proper

In [6]:
# Create empty lists to store information
job_titles = []
company_names = []
company_location = []
post_dates = []
applicants = []
job_desc = []

# Visit each link one by one to scrape the information
print('Visiting the links and collecting information just started.')
for i, link in enumerate(links):
    try:
        driver.get(link)
        time.sleep(2)
        
        # Click See More Button
        see_more_button = driver.find_element(By.XPATH, f"//button[@aria-label='Click to see more description']")
        driver.execute_script("arguments[0].click();", see_more_button)
        time.sleep(2)
    except Exception as e:
        print(e)
    
    # Find the general information of the job offers
    contents = driver.find_elements(By.CLASS_NAME, 'p5')
    for content in contents:
        try:
            title = content.find_element(By.CLASS_NAME, 'job-details-jobs-unified-top-card__job-title').text
            job_titles.append(title)

            details = content.find_element(By.CLASS_NAME, 'job-details-jobs-unified-top-card__primary-description').text.split('·')
            print(details)   
            
            company_names.append(details[0])
            
            # Extract company_location and post_dates directly
            company_location_and_post_date = details[1].strip()
            company_location.append(company_location_and_post_date)
            
            # Split company_location_and_post_date into location and post_dates
            split_details = re.split(r'(\d+)', company_location_and_post_date)
            split_details = [part.strip() for part in split_details if part.strip()]
            
            print("Split details: ", split_details)
            
            # Separate lists for the extracted items
            if len(split_details) == 3:
                company_location.append(split_details[0])
                
                combining_post_dates = split_details[1] + split_details[2]
                
                #post_dates.append(split_details[1])
                post_dates.append(combining_post_dates)
            else:
                # Handle the case where there's no post date
                company_location.append(split_details[0])
                post_dates.append("N/A")
            
            applicants.append(details[2])
            
            print(company_names)
            print(company_location)
            print(post_dates)
            print(applicants)
            
            print(f'Scraping the Job Offer {i + 1} DONE.')
        
        except Exception as e:
            print(e)
            
        time.sleep(2)
    
    # Scraping the job description
    job_description = driver.find_elements(By.CLASS_NAME, 'jobs-description__content')
    for description in job_description:
        try:
            job_text = description.find_element(By.CLASS_NAME, "jobs-box__html-content").text
            job_desc.append(job_text)
            print(f'Scraping the Job Description {i + 1}')
        except Exception as e:
            print(e)
        time.sleep(2)

# Creating the dataframe
df = pd.DataFrame(list(zip(job_titles, company_names, company_location, post_dates, applicants, job_desc)), columns=['Job Title', 'Company Name', 'Company Location', 'Post Dates', 'Number of Applicants', 'Job Description'])

Visiting the links and collecting information just started.
['Sense Software Solutions ', ' Philippines Reposted  2 weeks ago  ', ' 88 applicants']
Split details:  ['Philippines Reposted', '2', 'weeks ago']
['Sense Software Solutions ']
['Philippines Reposted  2 weeks ago', 'Philippines Reposted']
['2weeks ago']
[' 88 applicants']
Scraping the Job Offer 1 DONE.
Scraping the Job Description 1
['Business Solution Partners ', ' Philippines  1 month ago  ', ' 82 applicants']
Split details:  ['Philippines', '1', 'month ago']
['Sense Software Solutions ', 'Business Solution Partners ']
['Philippines Reposted  2 weeks ago', 'Philippines Reposted', 'Philippines  1 month ago', 'Philippines']
['2weeks ago', '1month ago']
[' 88 applicants', ' 82 applicants']
Scraping the Job Offer 2 DONE.
Scraping the Job Description 2
['Turing ', ' Philippines  2 months ago  ', ' 27 applicants']
Split details:  ['Philippines', '2', 'months ago']
['Sense Software Solutions ', 'Business Solution Partners ', 'Turin

Scraping the Job Description 10
['VIPRE Security Group ', ' Philippines Reposted  2 weeks ago  ', ' 216 applicants']
Split details:  ['Philippines Reposted', '2', 'weeks ago']
['Sense Software Solutions ', 'Business Solution Partners ', 'Turing ', 'Career Shepherd ', 'Sourcepass ', 'Sourcepass ', 'Fingerprint for Success (F4S) ', 'Filta. ', '3Cloud ', 'Asurion ', 'VIPRE Security Group ']
['Philippines Reposted  2 weeks ago', 'Philippines Reposted', 'Philippines  1 month ago', 'Philippines', 'Philippines  2 months ago', 'Philippines', 'Manila, National Capital Region, Philippines  1 month ago', 'Manila, National Capital Region, Philippines', 'Metro Manila  1 month ago', 'Metro Manila', 'Metro Manila  2 months ago', 'Metro Manila', 'Philippines  3 weeks ago', 'Philippines', 'Manila, National Capital Region, Philippines  4 weeks ago', 'Manila, National Capital Region, Philippines', 'Philippines Reposted  1 week ago', 'Philippines Reposted', 'Taguig, National Capital Region, Philippines Re

Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-details-jobs-unified-top-card__job-title"}
  (Session info: chrome=118.0.5993.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104301e08 chromedriver + 5025288
1   chromedriver                        0x00000001042f8c23 chromedriver + 4987939
2   chromedriver                        0x0000000103e9ae67 chromedriver + 409191
3   chromedriver                        0x0000000103eea1b9 chromedriver + 733625
4   chromedriver                        0x0000000103eea371 chromedriver + 734065
5   chromedriver                        0x0000000103eddb66 chromedriver + 682854
6   chromedriver                        0x0000000103f1150d chromedriver + 894221
7   chromedriver                        0x0000000103edda28 chromedriver + 682536
8   chromedriver

Scraping the Job Description 18
['OffSec ', ' Philippines Reposted  1 week ago  ', ' 141 applicants']
Split details:  ['Philippines Reposted', '1', 'week ago']
['Sense Software Solutions ', 'Business Solution Partners ', 'Turing ', 'Career Shepherd ', 'Sourcepass ', 'Sourcepass ', 'Fingerprint for Success (F4S) ', 'Filta. ', '3Cloud ', 'Asurion ', 'VIPRE Security Group ', '3Cloud ', 'Eclaro ', 'RealPage, Inc. ', 'WebFX ', 'Nextiva ', 'Netrix Global ', 'Security Bank Corporation ', 'OffSec ']
['Philippines Reposted  2 weeks ago', 'Philippines Reposted', 'Philippines  1 month ago', 'Philippines', 'Philippines  2 months ago', 'Philippines', 'Manila, National Capital Region, Philippines  1 month ago', 'Manila, National Capital Region, Philippines', 'Metro Manila  1 month ago', 'Metro Manila', 'Metro Manila  2 months ago', 'Metro Manila', 'Philippines  3 weeks ago', 'Philippines', 'Manila, National Capital Region, Philippines  4 weeks ago', 'Manila, National Capital Region, Philippines', 'P

Scraping the Job Description 21
['OffSec ', ' Philippines Reposted  1 week ago  ', ' 337 applicants']
Split details:  ['Philippines Reposted', '1', 'week ago']
['Sense Software Solutions ', 'Business Solution Partners ', 'Turing ', 'Career Shepherd ', 'Sourcepass ', 'Sourcepass ', 'Fingerprint for Success (F4S) ', 'Filta. ', '3Cloud ', 'Asurion ', 'VIPRE Security Group ', '3Cloud ', 'Eclaro ', 'RealPage, Inc. ', 'WebFX ', 'Nextiva ', 'Netrix Global ', 'Security Bank Corporation ', 'OffSec ', 'Quantrics Enterprises Inc. ', '3Cloud ', 'OffSec ']
['Philippines Reposted  2 weeks ago', 'Philippines Reposted', 'Philippines  1 month ago', 'Philippines', 'Philippines  2 months ago', 'Philippines', 'Manila, National Capital Region, Philippines  1 month ago', 'Manila, National Capital Region, Philippines', 'Metro Manila  1 month ago', 'Metro Manila', 'Metro Manila  2 months ago', 'Metro Manila', 'Philippines  3 weeks ago', 'Philippines', 'Manila, National Capital Region, Philippines  4 weeks ago

...

['Infinera ', ' Philippines  2 weeks ago  ', ' 24 applicants']
Split details:  ['Philippines', '2', 'weeks ago']
['Sense Software Solutions ', 'Business Solution Partners ', 'Turing ', 'Career Shepherd ', 'Sourcepass ', 'Sourcepass ', 'Fingerprint for Success (F4S) ', 'Filta. ', '3Cloud ', 'Asurion ', 'VIPRE Security Group ', '3Cloud ', 'Eclaro ', 'RealPage, Inc. ', 'WebFX ', 'Nextiva ', 'Netrix Global ', 'Security Bank Corporation ', 'OffSec ', 'Quantrics Enterprises Inc. ', '3Cloud ', 'OffSec ', 'Vita Green ', 'MultiplyMii ', 'Quantrics Enterprises Inc. ', 'GoTeam ', 'Inspectorio ', 'GoTeam ', 'Quantrics Enterprises Inc. ', 'Hue Lens ', 'WebFX ', 'Quantrics Enterprises Inc. ', 'Infinera ']
['Philippines Reposted  2 weeks ago', 'Philippines Reposted', 'Philippines  1 month ago', 'Philippines', 'Philippines  2 months ago', 'Philippines', 'Manila, National Capital Region, Philippines  1 month ago', 'Manila, National Capital Region, Philippines', 'Metro Manila  1 month ago', 'Metro Manil

Scraping the Job Description 34
['3Cloud ', ' Philippines Reposted  2 weeks ago  ', ' 147 applicants']
Split details:  ['Philippines Reposted', '2', 'weeks ago']
['Sense Software Solutions ', 'Business Solution Partners ', 'Turing ', 'Career Shepherd ', 'Sourcepass ', 'Sourcepass ', 'Fingerprint for Success (F4S) ', 'Filta. ', '3Cloud ', 'Asurion ', 'VIPRE Security Group ', '3Cloud ', 'Eclaro ', 'RealPage, Inc. ', 'WebFX ', 'Nextiva ', 'Netrix Global ', 'Security Bank Corporation ', 'OffSec ', 'Quantrics Enterprises Inc. ', '3Cloud ', 'OffSec ', 'Vita Green ', 'MultiplyMii ', 'Quantrics Enterprises Inc. ', 'GoTeam ', 'Inspectorio ', 'GoTeam ', 'Quantrics Enterprises Inc. ', 'Hue Lens ', 'WebFX ', 'Quantrics Enterprises Inc. ', 'Infinera ', 'Quantrics Enterprises Inc. ', '3Cloud ']
['Philippines Reposted  2 weeks ago', 'Philippines Reposted', 'Philippines  1 month ago', 'Philippines', 'Philippines  2 months ago', 'Philippines', 'Manila, National Capital Region, Philippines  1 month ago'

Scraping the Job Description 36
['Michael Page ', ' Philippines  11 hours ago  ', ' 21 applicants']
Split details:  ['Philippines', '11', 'hours ago']
['Sense Software Solutions ', 'Business Solution Partners ', 'Turing ', 'Career Shepherd ', 'Sourcepass ', 'Sourcepass ', 'Fingerprint for Success (F4S) ', 'Filta. ', '3Cloud ', 'Asurion ', 'VIPRE Security Group ', '3Cloud ', 'Eclaro ', 'RealPage, Inc. ', 'WebFX ', 'Nextiva ', 'Netrix Global ', 'Security Bank Corporation ', 'OffSec ', 'Quantrics Enterprises Inc. ', '3Cloud ', 'OffSec ', 'Vita Green ', 'MultiplyMii ', 'Quantrics Enterprises Inc. ', 'GoTeam ', 'Inspectorio ', 'GoTeam ', 'Quantrics Enterprises Inc. ', 'Hue Lens ', 'WebFX ', 'Quantrics Enterprises Inc. ', 'Infinera ', 'Quantrics Enterprises Inc. ', '3Cloud ', 'WebFX ', 'Michael Page ']
['Philippines Reposted  2 weeks ago', 'Philippines Reposted', 'Philippines  1 month ago', 'Philippines', 'Philippines  2 months ago', 'Philippines', 'Manila, National Capital Region, Philippin

Scraping the Job Description 38
['Inspectorio ', ' Philippines  1 week ago  ', ' 113 applicants']
Split details:  ['Philippines', '1', 'week ago']
['Sense Software Solutions ', 'Business Solution Partners ', 'Turing ', 'Career Shepherd ', 'Sourcepass ', 'Sourcepass ', 'Fingerprint for Success (F4S) ', 'Filta. ', '3Cloud ', 'Asurion ', 'VIPRE Security Group ', '3Cloud ', 'Eclaro ', 'RealPage, Inc. ', 'WebFX ', 'Nextiva ', 'Netrix Global ', 'Security Bank Corporation ', 'OffSec ', 'Quantrics Enterprises Inc. ', '3Cloud ', 'OffSec ', 'Vita Green ', 'MultiplyMii ', 'Quantrics Enterprises Inc. ', 'GoTeam ', 'Inspectorio ', 'GoTeam ', 'Quantrics Enterprises Inc. ', 'Hue Lens ', 'WebFX ', 'Quantrics Enterprises Inc. ', 'Infinera ', 'Quantrics Enterprises Inc. ', '3Cloud ', 'WebFX ', 'Michael Page ', 'Turing ', 'Inspectorio ']
['Philippines Reposted  2 weeks ago', 'Philippines Reposted', 'Philippines  1 month ago', 'Philippines', 'Philippines  2 months ago', 'Philippines', 'Manila, National Ca

Scraping the Job Description 40


## Storing Collected Jobs Data in a CSV File

In [7]:
# Storing the data to a CSV file
df.to_csv('startup_jobs_entrylevel.csv', index=False)

# Output job descriptions to a TXT file
with open('startups_job_descriptions.txt', 'w', encoding="utf-8") as f:
    for line in job_desc:
        f.write(line)
        f.write('\n')

# Close the WebDriver
driver.quit()

In [8]:
df

,Job Title,Company Name,Company Location,Post Dates,Number of Applicants,Job Description
0,PowerBI Developer,Sense Software Solutions,Philippines Reposted 2 weeks ago,2weeks ago,88 applicants,About the job\nTechnical :\n4+ years in PowerB...
1,NetSuite SuiteScript Software Engineer,Business Solution Partners,Philippines Reposted,1month ago,82 applicants,About the job\nLocation: Home Office (US based...
2,Flutter Engineer,Turing,Philippines 1 month ago,2months ago,27 applicants,About the job\nA rapidly-growing company that ...
3,Flutter Developer (For Active Pooling),Career Shepherd,Philippines,1month ago,291 applicants,About the job\nDISCLAIMER: This job posting is...
4,iOS Mobile Application Developer,Sourcepass,Philippines 2 months ago,1month ago,19 applicants,About the job\nAbout Sourcepass\n\nWe are Sour...
5,Android Mobile Application Developer,Sourcepass,Philippines,2months ago,125 applicants,About the job\nAbout Sourcepass\n\nWe are Sour...
6,Expression of Interest: Mobile Application Dev...,Fingerprint for Success (F4S),"Manila, National Capital Region, Philippines ...",3weeks ago,42 applicants,About the job\nWe are inviting professionals i...
7,Mid-level Frontend Developer – with Laptop & I...,Filta.,"Manila, National Capital Region, Philippines",4weeks ago,36 applicants,About the job\n‘Invent better outcomes’ with t...
8,DevOps Engineer (Infra/SRE),3Cloud,Metro Manila 1 month ago,1week ago,237 applicants,About the job\nAre you looking for a role that...
9,Software Engineer 2,Asurion,Metro Manila,5days ago,12 applicants,About the job\nWe are looking for a team membe...
